In [18]:
# load basics library 
import pandas as pd, numpy as np

# import pandasql library to work with dataframe with SQL 
from pandasql import sqldf

pysqldf = lambda q: sqldf(q, globals())

In [19]:
# example df
# random integer >= 1 and <= 20, with 5 counts 

#name=['']
df = pd.DataFrame({'name':['jim','sam','amy','liz','jane',
                          'jim','sam','amy','liz','jane'],
                   'point':np.random.randint(1,20,10),
                   'cash':np.random.randint(1,20,10),
                   'pay':np.random.randint(1,20,10)} )
df= df[['name','cash','pay','point']]

In [20]:
df.head(3)

,name,cash,pay,point
0,jim,6,5,7
1,sam,9,5,17
2,amy,4,1,4


### 1)  product all value on every category  


In [21]:
# http://www.itpub.net/thread-1577037-1-1.html

# METHOD 1 : via SQL 
SQL2="""

SELECT name,
       exp(s_log_cash) AS sum_cash
FROM
  ( SELECT name,
           sum(log(cash)) AS s_log_cash
   FROM df
   GROUP BY 1) sub

"""

In [22]:
# METHOD 2 : via pandas 

def get_log(x):
    return np.log(x)

def get_exp(x):
    return np.exp(x)


# group over index 
df.set_index('name').apply(get_log).groupby(level=0).sum().apply(get_exp)

,cash,pay,point
name,,,
amy,20.0,1.0,8.0
jane,289.0,342.0,30.0
jim,18.0,15.0,7.0
liz,90.0,68.0,44.0
sam,72.0,55.0,102.0


In [23]:
df.set_index('name').apply(lambda x : np.log(x))\
                    .groupby(level=0).sum()\
                    .apply(lambda x : np.exp(x))

,cash,pay,point
name,,,
amy,20.0,1.0,8.0
jane,289.0,342.0,30.0
jim,18.0,15.0,7.0
liz,90.0,68.0,44.0
sam,72.0,55.0,102.0


In [7]:
#SELECT x.val from data x, data y
#GROUP BY x.val
#HAVING SUM(SIGN(1-SIGN(y.val-x.val)))/COUNT(*) > .5
#LIMIT 1

In [24]:

SQL3="""


SELECT  name,
        AVG(cash)
FROM (SELECT 
        name,
        cash
      FROM df
      ORDER BY name
      LIMIT 200 - (SELECT COUNT(*) FROM df) % 2   
      OFFSET (SELECT (COUNT(*) - 1) / 2
              FROM df))
              group by 1 
"""

SQL4="""

SELECT x.cash 
FROM   (SELECT 
               cash, 
               Count(1) OVER (partition BY name)        AS TotalRows, 
               Row_number() OVER (ORDER BY cash ASC) AS AmountOrder 
        FROM   df
        group by 1) x 
WHERE  x.AmountOrder = Round(x.TotalRows / 2.0, 0) 

"""

In [40]:
#pysqldf(SQL4)
#df
#df[df.name=='liz']#.describe()

### 2) Second largest number

In [46]:
# https://leetcode.com/articles/second-highest-salary/

"""
+----+--------+
| Id | Salary |
+----+--------+
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |
+----+--------+

"""

df_x = pd.DataFrame({'id':[1,2,3],'salary':[100,200,300]})

In [33]:
df_x

,id,salary
0,1,100
1,2,200
2,3,300


In [44]:
sql_6="""

select 
ifnull(a.salary,0) as SecondHighestSalary
from 
df_x a 
where 
a.salary  < (select max(b.salary) from df_x b )
order by 1 desc
limit 1 

"""

sql_6_="""

SELECT
    IFNULL(
      (SELECT DISTINCT Salary
       FROM df_x
       ORDER BY Salary DESC
        LIMIT 1 OFFSET 1),
    NULL) AS SecondHighestSalary

"""

In [45]:
pysqldf(sql_6)

,SecondHighestSalary
0,200
